In [16]:
import os
import sys
code_dir = os.path.split(os.getcwd())[0]
if code_dir not in sys.path:
    sys.path.append(code_dir)

In [17]:
import numpy as np
import pickle

# my code
from contrastive_divergence_optimiser import CDOptimiser
from distribution import RBMLatentPosterior, MultivariateBernoulliNoise, ChowLiuTree
from latent_nce_optimiser import LatentNCEOptimiser
from latent_variable_model import RestrictedBoltzmannMachine
from utils import plot_log_likelihood_training_curves, average_log_likelihood

from copy import deepcopy
from itertools import product
from matplotlib import pyplot as plt
from matplotlib import rc
from numpy import random as rnd
from pandas import DataFrame
from pomegranate import BayesianNetwork

%matplotlib inline
rc('lines', linewidth=1)
rc('font', size=18)
rc('legend', fontsize=16)
rc('text', usetex=True)

# For reproducibility
rng = rnd.RandomState(1083463236)

In [18]:
exp_name = 'partial-m-step-with-cg-nz=10'  # CHANGE ME

exp_res_dir = '/afs/inf.ed.ac.uk/user/s17/s1771906/masters-project/ben-rhodes-masters-project/experimental-results/'
save_res_dir = '/afs/inf.ed.ac.uk/user/s17/s1771906/masters-project/ben-rhodes-masters-project/proposal/experiments/'
exp_dir = os.path.join(exp_res_dir, exp_name)
save_dir = os.path.join(save_res_dir, exp_name)

## calculate log-likes

In [19]:
for i, file in enumerate(os.listdir(exp_dir)):
    exp = os.path.join(exp_dir, file)
    
    optimiser = pickle.load(open(os.path.join(exp, 'lnce_optimiser.p'), 'rb'))
    cd_optimiser = pickle.load(open(os.path.join(exp, 'cd_optimiser.p'), 'rb'))
    nce_optimiser = pickle.load(open(os.path.join(exp, 'nce_optimiser.p'), 'rb'))
    
    loaded_data = np.load(os.path.join(exp, 'data.npz'))
    loaded_truth = np.load(os.path.join(exp, 'true_weights_and_likelihood.npz'))
    loaded_init = np.load(os.path.join(exp, 'init_theta_and_likelihood.npz'))
    loaded_lnce = np.load(os.path.join(exp, 'lnce_results.npz'))
    loaded_cd = np.load(os.path.join(exp, 'cd_results.npz'))
    loaded_nce = np.load(os.path.join(exp, 'nce_results.npz'))
    loaded_true = np.load(os.path.join(exp, 'true_weights_and_likelihood.npz'))
    
    X = loaded_data['X']
    Y = loaded_data['Y']

    true_theta = loaded_truth['true_theta']
    true_log_like = loaded_truth['ll']
    
    theta0 = loaded_init['theta0']
    init_log_like = loaded_init['ll']
    
    lnce_thetas = loaded_lnce['params']
    J1s = loaded_lnce['J1s']
    lnce_times = loaded_lnce['times']
    av_log_like_lnce = loaded_lnce['ll']
    
    cd_thetas = loaded_cd['params']
    cd_times = loaded_cd['times']
    av_log_like_cd = loaded_cd['ll']
    
    nce_thetas = loaded_nce['params']
    Js = loaded_nce['Js']
    nce_times = loaded_nce['times']
    av_log_like_nce = loaded_nce['ll']
    Js_for_lnce_thetas = loaded_nce['Js_for_lnce_thetas']

    ####### COMPUTE LOG-LIKELIHOODS ###########
    lnce_time_ids = np.array(sorted(list(set(np.exp(np.linspace(0, np.log(len(optimiser.times)))).astype(int)))))
    optimiser.times = optimiser.times[lnce_time_ids]
    optimiser.thetas = optimiser.thetas[lnce_time_ids]
    optimiser.J1s = optimiser.J1s[lnce_time_ids]
    av_log_like_lnce = optimiser.av_log_like_for_each_iter(X)
    
    # av_log_like_cd = cd_optimiser.av_log_like_for_each_iter(X)
    
    nce_time_ids = np.array(sorted(list(set(np.exp(np.linspace(0, np.log(len(nce_optimiser.times)))).astype(int)))))
    nce_optimiser.times = nce_optimiser.times[nce_time_ids]
    nce_optimiser.thetas = nce_optimiser.thetas[nce_time_ids]
    nce_optimiser.Js = nce_optimiser.Js[nce_time_ids]
    av_log_like_nce = nce_optimiser.av_log_like_for_each_iter(X)

    true_log_like = average_log_likelihood(true_data_dist, X)
    init_log_like = average_log_likelihood(init_model, X)
    
    X_mean = np.mean(X, axis=0)
    noise_dist = MultivariateBernoulliNoise(X_mean)
    noise_log_like = np.mean(np.log(noise_dist(X)))
    
    np.savez(os.path.join(exp, "data"), X=X, Y=optimiser.Y)
    np.savez(os.path.join(exp, "true_weights_and_likelihood"), true_theta=true_theta, ll=true_log_like)
    np.savez(os.path.join(exp, "init_theta_and_likelihood"), theta0=theta0, ll=init_log_like)
    np.savez(os.path.join(exp, "lnce_results"), params=optimiser.thetas, J1s=optimiser.J1s,
             times=optimiser.times, ll=av_log_like_lnce)
    np.savez(os.path.join(exp, "cd_results"), params=cd_optimiser.thetas, times=cd_optimiser.times, ll=av_log_like_cd)
    np.savez(os.path.join(exp, "nce_results"), params=nce_optimiser.thetas, times=nce_optimiser.times,
             Js=nce_optimiser.Js, Js_for_lnce_thetas=Js_for_lnce_thetas, ll=av_log_like_nce)

FileNotFoundError: [Errno 2] No such file or directory: '/afs/inf.ed.ac.uk/user/s17/s1771906/masters-project/ben-rhodes-masters-project/experimental-results/partial-m-step-with-cg-nz=10/1/lnce_optimiser.p'